In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import *
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import BaggingClassifier
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from load_data import get_data

In [2]:
x_train, y_train, x_pred = get_data()

C:\Users\gango\Projects\ADA\load_data.py:29: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  alldf = pd.concat([train_df,test_df])


Training set size: (24584, 57)


In [3]:
from sklearn.linear_model import LogisticRegression, RidgeClassifier, Lasso, ElasticNet, Lars, LassoLars, OrthogonalMatchingPursuit, BayesianRidge, SGDClassifier

In [4]:
def train_model(model):
    cv_results = cross_validate(BaggingClassifier(model, verbose=5),
                                x_train, y_train,
                                scoring=['roc_auc'], cv=5, n_jobs=4)
    auc = cv_results['test_roc_auc'].mean()
    print(f"{type(model)}: {auc}")
    return auc

In [6]:
%%time
models = [LogisticRegression(), RidgeClassifier()] #, ElasticNet(), Lars(), LassoLars(), OrthogonalMatchingPursuit(), BayesianRidge(), SGDClassifier()] # , Lasso()
linera_models_res = {type(model): train_model(model) for model in models}

<class 'sklearn.linear_model._logistic.LogisticRegression'>: 0.8680296806286465
<class 'sklearn.linear_model._ridge.RidgeClassifier'>: 0.8675468285980221
Wall time: 10.7 s


In [7]:
from sklearn.neighbors import *

In [8]:
%%time
models = [KNeighborsClassifier()]
neighbor_models_res = {type(model): train_model(model) for model in models}

<class 'sklearn.neighbors._classification.KNeighborsClassifier'>: 0.8321787503724376
Wall time: 3min 9s


In [9]:
from sklearn.tree import *

In [10]:
%%time
models = [DecisionTreeClassifier()]
tree_models_res = {type(model): train_model(model) for model in models}

<class 'sklearn.tree._classes.DecisionTreeClassifier'>: 0.8575384945015202
Wall time: 14.5 s


In [11]:
all_res = {}
for res in [linera_models_res, neighbor_models_res, tree_models_res]:
    all_res.update(res)

In [12]:
names = list(all_res.keys())
aucs = list(all_res.values())

pd.DataFrame({'name':names, 'auc': aucs}).to_csv('overview_bagging_auc.csv', index=False)

In [ ]:
param_grid = {'n_estimators': [10, 50, 100], 'max_depth': [14]}
clf = GridSearchCV(RandomForestClassifier(),
                   scoring='roc_auc',
                   param_grid=param_grid, cv=5, n_jobs=4)

print(clf.fit(x_train, y_train))

In [ ]:
best_i = np.argmax(clf.cv_results_['mean_test_score'])
print(f"Best auc: {np.max(clf.cv_results_['mean_test_score'])}")
print(f"Best params: {clf.cv_results_['params'][best_i]}")

In [ ]:
plt.imshow(clf.cv_results_['mean_test_score'])

In [ ]:
clf.cv_results_['params']

In [ ]:
pred = clf.predict_proba(x_train)[:, 1]
roc_auc_score(y_train, pred)

In [ ]:
y_train.to_numpy().astype(np.int).shape, pred.shape

In [ ]:
# aucs = []
# max_depths = []
# for max_depth in range(2,16):
#     cv_results = cross_validate(RandomForestClassifier(max_depth=max_depth),
#                                 x_train, y_train,
#                                 scoring=['roc_auc', 'accuracy'], cv=5)
#     auc = cv_results['test_roc_auc'].mean()
#     aucs.append(auc)
#     max_depths.append(max_depth)
#     print(f"Fa melyseg: {max_depth}\t{auc},\t{cv_results['test_accuracy'].mean()}")

In [ ]:
cv_results = cross_validate(DecisionTreeClassifier(max_depth = 14),
                            x_train, y_train,
                            scoring=['roc_auc', 'accuracy'], cv=5, n_jobs=-1)
auc = cv_results['test_roc_auc'].mean()
print(f"\t{auc},\t{cv_results['test_accuracy'].mean()}")

In [ ]:
# plt.plot(max_depths, aucs)
# plt.show()
# print(f"Best max_depth: {max_depths[np.argmax(aucs)]}")
# print(f"Best auc: {np.max(aucs)}")

In [ ]:
model = ExtraTreesClassifier(max_depth = 14, n_estimators = 1000)
model.fit(x_train, y_train)
pred = model.predict_proba(x_pred)
# accuracy_score(ismert_df[target],model.predict(ismert_df[bemeno_valtozok]))

In [ ]:
res_df = pd.DataFrame({'session_id': cust_df[cust_df['test_or_train_flag'] == 1].session_id, 'prob': pred[:, 1]})
res_df.to_csv('res.csv', index=False)